In [3]:
import numpy as npy
import cvxpy as cvpy

# Exchange rate data.
tickers = ["USD", "EUR", "GBP", "CAD", "JPY", "CNY", "RUB", "MXN", "INR", "BRL"]
n = len(tickers)
F = npy.zeros((n, n))

# USD
data = ([1.0, 0.87, 0.76, 1.31, 108.90, 6.72, 65.45, 19.11, 71.13, 3.69],
# EUR
[1.0, 0.88, 1.51, 125.15, 7.72, 75.23, 21.96, 81.85, 4.24],
# GBP
[1.0, 1.72, 142.94, 8.82, 85.90, 25.08, 93.50, 4.84],
# CAD
[1.0, 82.93, 5.11, 49.82, 14.54, 54.23, 2.81],
# JPY
[1.0, 0.062, 0.60, 0.18, 0.65, 0.034],
# CNY
[1.0, 9.74, 2.85, 10.61, 0.55],
# RUB
[1.0, 0.29, 1.09, 0.056],
# MXN
[1.0, 3.73, 0.19],
# INR
[1.0, 0.052],
# BRL
[1.0])

for i in range(n):
    F[i,i:] = data[i]
for j in range(n):
    for i in range(j+1,n):
        F[i,j] = 1.035/F[j,i]
        
# Initial and final portfolios
c_req = npy.arange(1,n+1)
c_req = 1e4*c_req/c_req.sum()
c_init = c_req[::-1]

In [4]:
# Decision variable (Here, it is a n*n matrix)
X = cvpy.Variable((n,n))

#Expression for c_final is obtained as follows 
#In cvxpy, we have the transpose atomic function (expr.T)
#From numpy, we have a matrix/list which is initialized to 1 for all elements

c_final = (X/F)@npy.ones(n) - X.T@npy.ones(n) + c_init

# calculating the final cost expression (penalty of the problem)
temp = []

for i in range(n):
  temp.append(npy.sqrt(F[i,0]/F[0,i]))

matrix = []

while temp!=[]:
  matrix.append(temp[:n])
  temp = temp[n:]

cost = (c_init - c_final)@matrix

# objective of the problem  
objective = cvpy.Minimize(cost)

#constraints for our problem
constraints = [c_final >= c_req, cvpy.diag(X) == 0, X.T@npy.ones(n) <= c_init]
constraints += [X>=0]

# formulating our convex optimization problem
prob = cvpy.Problem(objective, constraints)

result = prob.solve()

print("The minimum cost is", result, "USD")

The minimum cost is 7.720059340057956 USD
